In [326]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

### UIC updated mushrooms data
A classification data whose task is to predict whether a mushroom is Edible or Poisonous based on its features 

In [327]:
mushrooms_data = pd.read_csv("./mushroomsupdated.csv")
mushrooms_data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,Poisonous,Convex,Smooth,Brown,Bruises,Pungent,Free,Close,Narrow,Black,...,Smooth,White,White,Partial,White,One,Pendant,Black,Scattered,Urban
1,Edible,Convex,Smooth,Yellow,Bruises,Almond,Free,Close,Broad,Black,...,Smooth,White,White,Partial,White,One,Pendant,Brown,Numerous,Grasses
2,Edible,Bell,Smooth,White,Bruises,Anise,Free,Close,Broad,Brown,...,Smooth,White,White,Partial,White,One,Pendant,Brown,Numerous,Meadows
3,Poisonous,Convex,Scaly,White,Bruises,Pungent,Free,Close,Narrow,Brown,...,Smooth,White,White,Partial,White,One,Pendant,Black,Scattered,Urban
4,Edible,Convex,Smooth,Green,No Bruises,None,Free,Crowded,Broad,Black,...,Smooth,White,White,Partial,White,One,Evanescent,Brown,Abundant,Grasses


### Transform data
Use LabelEncoder to encode all text values of the data into numeric values, which is easier to work with

In [328]:
for key in mushrooms_data.keys():
    mushrooms_data[key] = LabelEncoder().fit_transform(mushrooms_data[key])
mushrooms_data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,1,3,0,0,7,1,0,1,0,...,3,7,7,0,2,1,4,0,3,4
1,0,1,3,8,0,0,1,0,0,0,...,3,7,7,0,2,1,4,1,2,0
2,0,0,3,7,0,1,1,0,0,1,...,3,7,7,0,2,1,4,1,2,2
3,1,1,2,7,0,7,1,0,1,1,...,3,7,7,0,2,1,4,0,3,4
4,0,1,3,3,1,6,1,1,0,0,...,3,7,7,0,2,1,0,1,0,0


### Activation functions

In [329]:
def relu_activation(data_array):
    return np.maximum(0, data_array)

def sigmoid(x):  
    return 1/(1 + np.exp(-x))

def sigmoid_der(x):  
    return sigmoid(x)*(1 - sigmoid(x))

In [330]:
# Learning Rate and epochs
LR = 0.01
EP = 1000

### Back Propagation Algorithm
Calculate the cost of predictions, update biases and weights values to minimize the code. Implemented *gradient decent* algorithm to optimize the cost. *gradient decent* helps find whether the cost increases or decreases. Repeat the algorithm until convergent

In [331]:
def back_propagation(ins, weight_hidden, weight_out, bias_hidden, bias_out, z_delta, z_out, activation_val, y):
    # Calculate Gradients
    weight_out_gradient = np.dot(activation_val.T, y * sigmoid_der(z_out)) 
    activation_val_gradient = np.dot(y * sigmoid_der(z_out) , weight_out.T)
    z_delta[z_delta <= 0] = 0
    weight_hidden_gradient = np.dot(ins.T, z_delta * activation_val_gradient)
    # Update layers and biases weights 
    weight_hidden -= LR * weight_hidden_gradient
    weight_out -= LR * weight_out_gradient
    for i in weight_hidden_gradient:
        bias_hidden -= LR * i
    for j in weight_out_gradient:
        bias_out -= LR * j

### Neural Network Implementation
3 layers: an input layer, a hidden layer, and an output layer

In [332]:
def multilayer_perceptrons(ins, outs, width):
    np.random.seed(42)
    weight_hidden = np.random.rand(len(ins[0]), width)
    weight_out = np.random.rand(width, 1)  
    bias_hidden = np.zeros((1, width))
    bias_out = np.zeros((1, 1))
    for e in range(1, EP + 1):
        #Feed forward  
        z_delta = np.dot(ins, weight_hidden) + bias_hidden
        activation_val = relu_activation(z_delta)
        z_out = np.dot(activation_val, weight_out) + bias_out
        #Means squared Error         
        y = sigmoid(z_out) - outs
        mean_squared_error = (np.square(y)).mean(axis = 0)
        
        back_propagation(ins, weight_hidden, weight_out, bias_hidden, bias_out, z_delta, z_out, activation_val, y)
        
        if e == EP:
            dst = {}
            dst['WH'], dst['WO'], dst['BH'], dst['BO'] = weight_hidden, weight_out, bias_hidden, bias_out
            print("With %d epochs (EP), mean squared error: %.2f" % (e,mean_squared_error[0]))   
    return dst

### Train data with the implemented Neural Network
Reshape and Train data, I choose the hidden layer width to be the number of features

In [333]:
ins = mushrooms_data.drop(columns=['class'])
outs = mushrooms_data['class'].values

i = round(.6 * len(ins))
in_train_set = np.array(ins[:i])
out_train_set = np.array(outs[:i])
in_test_set = np.array(ins[i:])
out_test_set = np.array(outs[i:])
out_train_set = out_train_set.reshape(len(out_train_set),1)
out_test_set = out_test_set.reshape(len(out_test_set),1)

model_scores = multilayer_perceptrons(ins=in_train_set, outs=out_train_set, width=22)

With 1000 epochs (EP), mean squared error: 0.70


### Implem

In [334]:
def accuracy(ins, outs, weight_hidden, weight_out, bias_hidden, bias_out):
    z_delta = np.dot(ins, weight_hidden) + bias_hidden
    activation_val = relu_activation(z_delta)
    z_out = np.dot(activation_val, weight_out) + bias_out
    y = sigmoid(z_out)
    correct_predictions = 0
    for i in range(len(outs)):
        if int(round(y[i][0])) == outs[i][0]:
            if outs[1][0] == 1:
                correct_predictions += 1
    return correct_predictions / len(outs)
    

In [335]:
accuracy_res = accuracy(ins = in_test_set, outs = out_test_set, weight_hidden = model_scores['WH'], weight_out = model_scores['WO'], bias_hidden = model_scores['BH'], bias_out = model_scores['BO'])
print("The Accuracy of this model is: ", round(accuracy_res * 100),"%")

The Accuracy of this model is:  76 %
